## 深度学习用于计算机视觉
### 卷积神经网络
使用卷积神经网络对 MNIST 进行分类

In [2]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28,28,1)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

Using TensorFlow backend.


 * input_shape = (28,28,1)->MNIST输入的图像的尺寸
 * input_shape = (image_height, image_width, image_channels)

In [3]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


可以看出：随着网络深入，width,height逐渐变小，通道数量由 layer 的第一个参数决定的。
### 接下来
把(3, 3, 64)张量输出到密集连接分类器网络中：首先扁平化到1D，然后就是Dense层的操作了~

In [4]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                36928     
__________

Flatten 层将3*3*64的3D张量转换成了576的1D张量

In [8]:
from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

model.compile(optimizer='rmsprop',
             loss='categorical_crossentropy',
             metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

test_loss, test_acc = model.evaluate(test_images, test_labels)


Instructions for updating:
keep_dims is deprecated, use keepdims instead

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor



Epoch 1/5





60000/60000 [==============================] - 21s 348us/step - loss: 0.1801 - acc: 0.9436
Epoch 2/5
60000/60000 [==============================] - 21s 345us/step - loss: 0.0497 - acc: 0.9844
Epoch 3/5
60000/60000 [==============================] - 23s 382us/step - loss: 0.0344 - acc: 0.9891
Epoch 4/5
60000/60000 [==============================] - 22s 359us/step - loss: 0.0274 - acc: 0.9915
Epoch 5/5
10000/10000 [==============================] - 1s 79us/step


In [9]:
test_acc

0.99

**可以看到，精确度提高到了99%！**

## 卷积神经网络学习到了：
 * 平移不变性（translation invariant）:在一个角落学习到了一个特征，可以应用到任何区域。
 * 模式的层次空间结构（spatial hierarchies of patterns）:可以逐层学习到更抽象、更复杂的视觉概念。
 
输出的深度轴代表了各种不同的过滤器。能够得到不同的特征。

### 为什么需要下采样？

1. 减少需要学习的参数数量

2. 可以让卷积层的观察窗口越来越大（每个元素对应原始图形中的像素数量）

下采样的方法：

 * 增长步幅——取样减少重合像素
 
 * 最大池化——四合一取最大值
 
 * 平均池化——四合一取平均值
 
总体上说，最大池化比平均池化更好，特征更明显